In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML：MLOps阶段2：实验：开始使用日志记录

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_logging.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
    <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_logging.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab上运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_with_logging.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在 Google Cloud 上使用 Vertex AI 进行端到端的 MLOps 生产环境。本教程涵盖了第二阶段：实验：开始使用日志记录。

### 目标

在本教程中，您将学习如何在与“Vertex AI”训练时使用Python和云日志记录。

本教程使用以下谷歌云机器学习服务：

- “云日志”

执行的步骤包括：

- 使用Python日志记录本地训练配置/结果。
- 使用Google云日志记录云存储中的训练配置/结果。

### 建议

在谷歌云上进行 E2E MLOps 时，在实验或正式训练模型时，以下是一些最佳实践，用于记录数据。

#### Python 记录

在本地进行临时培训时，请使用 Python 的记录包。

#### 云日志

在云上进行培训时，请使用 `Google Cloud 日志`。

#### 实验

在执行实验以比较不同实验配置的结果时，请结合使用 Vertex AI 实验和记录。

数据集

本教程不使用数据集。对示例数据集的引用仅用于演示目的。

### 成本
本教程使用 Google Cloud 的收费组件：

- Vertex AI

了解[Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-logging $USER_FLAG -q

### 重新启动内核

安装了额外的软件包后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个账户时，您将获得300美元的免费信用，可用于计算/存储的费用。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI、Compute Engine、Cloud Storage 和 Cloud Logging API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component,logging)。

1. 如果您在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保
Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并将以 `$` 为前缀的 Python 变量插入到这些命令中。

#### 设置您的项目 ID

**如果您不知道您的项目 ID**，您可以使用`gcloud`来获取您的项目 ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量在本笔记本的其余部分中使用。下面是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能在Vertex AI中使用多区域存储桶进行训练。并非所有区域都提供所有Vertex AI服务的支持。

了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在直播教程会话中，您可能正在使用共享测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您需要为每个实例会话创建一个时间戳，并将时间戳附加到您在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud账户

**如果您正在使用Vertex AI工作台笔记本**，则您的环境已经得到了验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并且按照提示进行操作，通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **点击创建服务帐号**。

2. 在**服务帐号名称**字段中输入名称，并点击**创建**。

3. 在**将此服务帐号授予项目访问权限**部分，点击角色下拉列表。在筛选框中输入“Vertex AI”，并选择**Vertex AI管理员**。在筛选框中输入“存储对象管理员”，并选择**存储对象管理员**。

4. 点击创建。包含您密钥的JSON文件会下载到本地环境。

5. 在下面的单元格中输入您服务帐号密钥的路径作为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库

In [ ]:
import logging

import google.cloud.aiplatform as aiplatform

初始化Python的Vertex AI SDK
为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## Python Logging

Python日志包被广泛用于Python脚本中的记录。常用功能包括：

- 设置日志级别。
- 将日志输出发送到控制台。
- 将日志输出发送到文件。

### Python日志中的日志级别

日志级别按顺序（从最低到最高）排列，每个级别都包括前一个级别：

1. 信息
2. 警告
3. 错误
4. 调试

默认情况下，日志级别设置为错误级别。

### 将日志输出发送到控制台

默认情况下，Python日志包将输出到控制台。请注意，在示例中，由于默认日志级别设置为错误，因此调试日志消息不会被输出。

In [ ]:
def logging_examples():
    logging.info("Model training started...")
    logging.warning("Using older version of package ...")
    logging.error("Training was terminated ...")
    logging.debug("Hyperparameters were ...")


logging_examples()

### 设置日志级别

要设置日志级别，您可以使用`getLogger()`来获取日志处理程序。您可以有多个日志处理程序。当不带任何参数调用`getLogger()`时，它会获取名为ROOT的默认处理程序。使用处理程序，您可以使用`setLevel()`方法设置日志级别。

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)

logging_examples()

清楚处理程序

有时，您可能希望重新配置日志记录。在这种情况下的一个常见做法是首先删除所有现有的日志处理程序，以进行重新开始。

In [ ]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

### 输出到本地文件

您可以使用`BasicConfig()`方法将日志输出保留到一个与Python脚本运行位置相对应的本地文件中，该方法接受以下参数：

- `filename`：要将日志输出写入的本地文件的文件路径。
- `level`：设置写入日志文件的日志级别。

*注意:* 您不能将Cloud Storage存储桶用作输出文件。

In [ ]:
logging.basicConfig(filename="mylog.log", level=logging.DEBUG)

logging_examples()

! cat mylog.log

使用Google Cloud Logging进行日志记录

您可以将日志输出保留并检索到`Google Cloud Logging`服务中。常用功能包括：

- 设置日志级别。
- 将日志输出发送到存储器。
- 从存储器中检索日志输出。

### 云日志中的日志级别

云日志中的日志级别顺序（从低到高）包括以下每个级别：

1. 信息
2. 警告
3. 错误
4. 调试

默认情况下，日志级别设置为警告级别。

### 可配置和存储日志数据

要使用`Google Cloud Logging`服务，请执行以下步骤：

1. 创建到该服务的客户端。
2. 获取该服务的处理程序。
3. 创建一个记录器实例并设置日志级别。
4. 将记录器实例附加到该服务。

了解更多关于[日志客户端库](https://cloud.google.com/logging/docs/reference/libraries)。

In [ ]:
import google.cloud.logging
from google.cloud.logging.handlers import CloudLoggingHandler

# Connect to the Cloud Logging service
cl_client = google.cloud.logging.Client(project=PROJECT_ID)
handler = CloudLoggingHandler(cl_client, name="mylog")

# Create a logger instance and logging level
cloud_logger = logging.getLogger("cloudLogger")
cloud_logger.setLevel(logging.INFO)

# Attach the logger instance to the service.
cloud_logger.addHandler(handler)

# Log something
cloud_logger.error("bad news")

日志输出

在特定级别的日志输出与 Python 日志记录方法和方法名称相同。唯一的区别是您在云日志记录器中使用您的实例代替日志。

In [ ]:
cloud_logger.info("Model training started...")
cloud_logger.warning("Using older version of package ...")
cloud_logger.error("Training was terminated ...")
cloud_logger.debug("Hyperparameters were ...")

获取日志条目

要获取已记录的输出，您需要：

1. 检索服务的日志句柄。
2. 使用句柄，调用方法 `list_entries()`。
3. 遍历条目。

In [ ]:
logger = cl_client.logger("mylog")

for entry in logger.list_entries():
    timestamp = entry.timestamp.isoformat()
    print("* {}: {}: {}".format(timestamp, entry.severity, entry.payload))

清理工作

要清理此项目中使用的所有Google Cloud资源，可以删除您在教程中使用的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。